## Считывание файла

In [61]:
import pyedflib
import numpy as np
import pandas as pd

In [63]:
def read_dataset(file_path):
    edf_file = pyedflib.EdfReader(file_path)
    n_signals = edf_file.signals_in_file
    signal_labels = edf_file.getSignalLabels()
    signals = [edf_file.readSignal(i) for i in range(n_signals)]
    edf_file.close()
    
    print('Сигналов обнаружено: ', n_signals)
    return signal_labels, signals

In [183]:
data_marked = pd.read_csv('marked_dataset.csv', index_col=0)
data = data_marked.drop(columns=['target'])
y_true = data_marked.target[:-6]
data.head()

,FrL,FrR,OcR
0,-0.121875,-0.163000,-0.078500
1,-0.140063,-0.188250,-0.059062
2,-0.079437,-0.178813,-0.004125
3,-0.030875,-0.097250,0.024625
4,0.006687,-0.040563,-0.005562


In [185]:
y_true

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
8640389   NaN
8640390   NaN
8640391   NaN
8640392   NaN
8640393   NaN
Name: target, Length: 8640394, dtype: float64

In [187]:
data.shape

(8640400, 3)

In [189]:
y_true

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
8640389   NaN
8640390   NaN
8640391   NaN
8640392   NaN
8640393   NaN
Name: target, Length: 8640394, dtype: float64

## Preprocessing

In [ ]:
def prepare_sequences_for_prediction(new_data, seq_length=7):
    sequences = []
    for i in range(len(new_data) - seq_length + 1):
        sequences.append(new_data[i:i + seq_length])
    return np.array(sequences)

seq_length = 7
X_new_seq = prepare_sequences_for_prediction(data, seq_length)

In [ ]:
X_new_seq.shape

## Модель

In [ ]:
# Загрузка модели
from tensorflow.keras.models import load_model
model = load_model('BiLSTM_512-256_model.h5')

y_pred_proba = model.predict(X_new_seq, batch_size=128)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred.shape

In [ ]:
most_probable_classes[most_probable_classes == 0].shape[0]

In [ ]:
most_probable_classes[most_probable_classes == 1].shape[0]

In [ ]:
most_probable_classes[most_probable_classes == 2].shape[0]

In [ ]:
X_new_seq.shape, y_new_pred.shape

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score

# Преобразуем y_true в numpy-массив, чтобы проще работать с None
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Исключаем элементы, где y_true равно None (или NaN, если данные изначально преобразуются)
mask = y_true != None  # Используем маску для фильтрации
filtered_y_true = y_true
filtered_y_pred = y_pred

precision_per_class = precision_score(filtered_y_true, filtered_y_pred, average=None, labels=[0, 1, 2])
recall_per_class = recall_score(filtered_y_true, filtered_y_pred, average=None, labels=[0, 1, 2])
f1_per_class = f1_score(filtered_y_true, filtered_y_pred, average=None, labels=[0, 1, 2])
accuracy = accuracy_score(filtered_y_true, filtered_y_pred)

# Вывод результатов
print("Метрики по каждому классу:")
for i in range(3):
    print(f"Класс {i}:")
    print(f"  Precision: {precision_per_class[i]:.2f}")
    print(f"  Recall: {recall_per_class[i]:.2f}")
    print(f"  F1-score: {f1_per_class[i]:.2f}")
print(f"\nОбщая Accuracy: {accuracy:.2f}")

In [ ]:
y_true

In [ ]:
y_new_pred